In [ ]:
import numpy as np
import pandas as pd
from pandas import crosstab
# import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_curve, roc_auc_score, auc, classification_report, confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, precision_recall_curve
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score, KFold, StratifiedKFold, cross_validate, train_test_split
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier, OutputCodeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from collections import Counter
from imblearn.over_sampling import SMOTE

pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 70)

Using TensorFlow backend.


In [2]:
df = pd.read_csv(r'C:\Users\1130j\Desktop\Yale\Data\all_recent_tweets_text_score_8_parsed_noDup.csv')

In [3]:
df = df.loc[df['classname'] != 'awareness']
df.loc[df['classname']=='None', 'class'] = 0
df.loc[df['classname']=='pain', 'class'] = 1
df.loc[df['classname'].isin(['pleasure', 'addiction']), 'class'] = 2

### Group Pleasure and Addiction together

In [4]:
crosstab(df['class'], df['classname'])

classname,None,addiction,pain,pleasure
class,,,,
0,166,0,0,0
1,0,0,229,0
2,0,197,0,572


### Train Test Split

In [65]:
train, test = train_test_split(df,test_size=0.3,random_state=0,stratify=df['class'])

In [66]:
num_features = train.columns.difference(['class', 'classname', 'user_id', 'screen_name'])
# num_features

In [67]:
# scaler = StandardScaler()
# train[num_features] = scaler.fit_transform(train[num_features])
# test[num_features] = scaler.fit_transform(test[num_features])

In [68]:
train[train.columns.difference(['user_id'])].groupby('class').mean()

,active,adjustment,adventurous,aggressive,agreeableness,ambitious,anxious,artistic,assertive,body_focus,cautious,cheerful,cold,conscientiousness,cooperative,depression,disciplined,dutiful,emotionally_aware,empathetic,energetic,extraversion,family_oriented,food_focus,friend_focus,friendly,generous,genuine,happiness,health_oriented,humble,imaginative,impulsive,independent,insecure,intellectual,leisure_oriented,liberal,melancholy,money_oriented,netspeak_focus,neuroticism,openness,organized,persuasive,power_driven,religion_oriented,reward_bias,self_assured,self_conscious,sexual_focus,sociable,social_skills,stressed,thinking_style,trusting,type_a,work_oriented,workhorse
class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,60.833320,6.498026,48.327608,56.242587,38.953222,51.073313,39.634743,44.992141,67.032800,42.750569,51.037259,43.770584,49.054788,51.997241,46.330993,82.019595,50.523656,51.627161,44.011413,43.549857,43.498594,53.000525,43.605299,43.243399,38.386965,38.646356,35.106537,48.762154,43.626146,40.737194,49.220178,39.956073,46.421767,60.114689,50.501746,62.606287,52.175939,43.288438,46.997139,54.260934,39.194855,47.480454,39.626117,50.482973,49.942719,55.913904,40.723838,47.054024,47.603678,40.420700,41.763522,41.287237,49.371440,49.894738,58.662853,43.219800,50.858576,54.417645,54.617230
1,53.697916,1.127901,38.282811,62.694849,36.972301,43.445246,39.417960,48.853396,70.330670,48.443002,62.588307,37.010522,60.426588,46.439228,44.660933,95.108609,46.908080,46.208051,50.722088,49.073934,38.838960,45.673089,48.180407,44.413125,38.112971,36.658225,37.918921,39.292041,34.973963,58.418405,53.268013,43.008039,47.468157,60.815753,60.630918,69.941501,34.467362,52.106015,57.965490,57.321898,27.564148,55.881004,44.650060,43.767790,35.230983,41.262853,38.599674,39.731306,50.178404,52.178351,48.373279,39.912062,42.444563,59.148834,49.423273,37.786994,54.010070,51.399985,46.537636
2,49.980886,1.822193,47.523295,71.237069,45.772476,42.503677,29.642229,44.741775,65.858204,67.975793,51.603146,34.421307,64.647241,30.631845,28.307760,92.924267,37.702341,31.632224,58.187581,54.950810,50.568276,42.016371,47.364321,51.846087,41.318373,25.241766,47.951027,52.792715,33.484266,49.724183,42.385072,47.728203,57.802610,57.386985,60.867564,63.261063,37.783905,59.313328,60.040113,51.541709,55.208100,56.829325,53.238457,32.052699,27.367778,38.343248,43.816979,48.633360,52.295815,42.977202,63.554982,34.522497,29.499466,55.865912,47.787977,41.552806,62.557664,36.510964,31.829844


In [69]:
X_train = train[train.columns.difference(['screen_name', 'user_id', 'classname', 'class'])]
y_train = train['class']
X_test = test[test.columns.difference(['screen_name', 'user_id', 'classname', 'class'])]
y_test = test['class']

### Baseline Model

In [70]:
lr = LogisticRegression(solver='newton-cg', random_state=0)
svc = SVC(random_state=0)
rf = RandomForestClassifier(random_state=0)
xgb = XGBClassifier(objective='multi:softmax', num_class=4, random_state=0)
nb = GaussianNB()

onevsone_lr = OneVsOneClassifier(lr).fit(X_train, y_train)
onevsone_svc = OneVsOneClassifier(svc).fit(X_train, y_train)
onevsone_rf = OneVsOneClassifier(rf).fit(X_train, y_train)
# onevsone_xgb = OneVsOneClassifier(xgb).fit(X_train, y_train)
onevsone_nb = OneVsOneClassifier(nb).fit(X_train, y_train)

onevsrest_lr = OneVsRestClassifier(lr).fit(X_train, y_train)
onevsrest_svc = OneVsRestClassifier(svc).fit(X_train, y_train)
onevsrest_rf = OneVsRestClassifier(rf).fit(X_train, y_train)
onevsrest_nb = OneVsRestClassifier(nb).fit(X_train, y_train)

# onevsone_lr = OneVsOneClassifier(lr).fit(X_train, y_train)
# onevsone_svc = OneVsOneClassifier(svc).fit(X_train, y_train)
# onevsone_rf = OneVsOneClassifier(rf).fit(X_train, y_train)
# # onevsone_xgb = OneVsOneClassifier(xgb).fit(X_train, y_train)

# onevsrest_lr = OneVsRestClassifier(lr).fit(X_train, y_train)
# onevsrest_svc = OneVsOneClassifier(svc).fit(X_train, y_train)
# onevsrest_rf = OneVsOneClassifier(rf).fit(X_train, y_train)
# onevsrest_xgb = OneVsOneClassifier(xgb).fit(X_train, y_train)

In [73]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6695121951219513.
ovo_lr: Test f1 micro is 0.6514285714285715
ovo_svc: Cross validation f1 micro is 0.6891147244805781.
ovo_svc: Test f1 micro is 0.6857142857142857
ovo_rf: Cross validation f1 micro is 0.6793134598012646.
ovo_rf: Test f1 micro is 0.6657142857142857
ovo_nb: Cross validation f1 micro is 0.610328214393255.
ovo_nb: Test f1 micro is 0.6114285714285714
ovr_lr: Cross validation f1 micro is 0.6767991568804577.
ovr_lr: Test f1 micro is 0.6542857142857142
ovr_svc: Cross validation f1 micro is 0.6805630834086118.
ovr_svc: Test f1 micro is 0.6857142857142857
ovr_rf: Cross validation f1 micro is 0.6768744354110207.
ovr_rf: Test f1 micro is 0.6771428571428572
ovr_nb: Cross validation f1 micro is 0.6238482384823849.
ovr_nb: Test f1 micro is 0.6085714285714285
xgb: Cross validation f1 micro is 0.6682174043962663.
xgb: Test f1 micro is 0.6885714285714286


In [77]:
mod = onevsrest_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.56      0.20      0.29        50
           1       0.45      0.22      0.29        69
           2       0.72      0.93      0.81       231

    accuracy                           0.69       350
   macro avg       0.58      0.45      0.47       350
weighted avg       0.64      0.69      0.64       350

micro avg                              0.69


### BIC 6

In [78]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven', 'health_oriented']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [79]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6707166516109606.
ovo_lr: Test f1 micro is 0.6714285714285714
ovo_svc: Cross validation f1 micro is 0.682987052092743.
ovo_svc: Test f1 micro is 0.6685714285714286
ovo_rf: Cross validation f1 micro is 0.6694218608852756.
ovo_rf: Test f1 micro is 0.6457142857142857
ovo_nb: Cross validation f1 micro is 0.6483739837398373.
ovo_nb: Test f1 micro is 0.6228571428571429
ovr_lr: Cross validation f1 micro is 0.6805630834086118.
ovr_lr: Test f1 micro is 0.6714285714285714
ovr_svc: Cross validation f1 micro is 0.6866606443842217.
ovr_svc: Test f1 micro is 0.66
ovr_rf: Cross validation f1 micro is 0.6682023486901535.
ovr_rf: Test f1 micro is 0.6514285714285715
ovr_nb: Cross validation f1 micro is 0.6532670882264378.
ovr_nb: Test f1 micro is 0.62
xgb: Cross validation f1 micro is 0.6719662752183078.
xgb: Test f1 micro is 0.6571428571428571


In [82]:
mod = onevsone_lr
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.38      0.18      0.24        50
           1       0.43      0.19      0.26        69
           2       0.72      0.92      0.81       231

    accuracy                           0.67       350
   macro avg       0.51      0.43      0.44       350
weighted avg       0.61      0.67      0.62       350

micro avg                              0.67


### BIC 10

In [83]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven',
                'health_oriented', 'adjustment', 'reward_bias', 'cheerful', 'leisure_oriented']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [84]:
# import math
# print('odds ratio    {}'.format(math.exp()) )
# print('odds ratio    {}'.format(math.exp())  )
# print('odds ratio    {}'.format(math.exp()) )
# print('odds ratio    {}'.format(math.exp()) )
# print('odds ratio    {}'.format(math.exp())  )

In [85]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6791779584462512.
ovo_lr: Test f1 micro is 0.6657142857142857
ovo_svc: Cross validation f1 micro is 0.6916139716952726.
ovo_svc: Test f1 micro is 0.6714285714285714
ovo_rf: Cross validation f1 micro is 0.6829569406805179.
ovo_rf: Test f1 micro is 0.66
ovo_nb: Cross validation f1 micro is 0.571017765733213.
ovo_nb: Test f1 micro is 0.5171428571428571
ovr_lr: Cross validation f1 micro is 0.680397470641373.
ovr_lr: Test f1 micro is 0.66
ovr_svc: Cross validation f1 micro is 0.6977416440831075.
ovr_svc: Test f1 micro is 0.6742857142857143
ovr_rf: Cross validation f1 micro is 0.6902890695573622.
ovr_rf: Test f1 micro is 0.6571428571428571
ovr_nb: Cross validation f1 micro is 0.6312556458897922.
ovr_nb: Test f1 micro is 0.5914285714285714
xgb: Cross validation f1 micro is 0.6792682926829269.
xgb: Test f1 micro is 0.64


In [88]:
mod = onevsone_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.60      0.12      0.20        50
           1       0.42      0.16      0.23        69
           2       0.69      0.94      0.80       231

    accuracy                           0.67       350
   macro avg       0.57      0.41      0.41       350
weighted avg       0.63      0.67      0.60       350

micro avg                              0.67


### BIC 15

In [89]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven',
                'health_oriented', 'adjustment', 'reward_bias', 'cheerful', 'leisure_oriented',
                'thinking_style', 'aggressive', 'organized', 'self_assured', 'religion_oriented']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [90]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.682836495031617.
ovo_lr: Test f1 micro is 0.6685714285714286
ovo_svc: Cross validation f1 micro is 0.6940228846732912.
ovo_svc: Test f1 micro is 0.6828571428571428
ovo_rf: Cross validation f1 micro is 0.6878801565793435.
ovo_rf: Test f1 micro is 0.6828571428571428
ovo_nb: Cross validation f1 micro is 0.587037037037037.
ovo_nb: Test f1 micro is 0.5828571428571429
ovr_lr: Cross validation f1 micro is 0.6865552544414333.
ovr_lr: Test f1 micro is 0.6742857142857143
ovr_svc: Cross validation f1 micro is 0.6903191809695874.
ovr_svc: Test f1 micro is 0.6828571428571428
ovr_rf: Cross validation f1 micro is 0.6829569406805179.
ovr_rf: Test f1 micro is 0.6657142857142857
ovr_nb: Cross validation f1 micro is 0.6324299909665763.
ovr_nb: Test f1 micro is 0.6057142857142858
xgb: Cross validation f1 micro is 0.6559620596205963.
xgb: Test f1 micro is 0.6514285714285715


In [94]:
mod = onevsone_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.73      0.16      0.26        50
           1       0.44      0.22      0.29        69
           2       0.71      0.94      0.81       231

    accuracy                           0.68       350
   macro avg       0.63      0.44      0.45       350
weighted avg       0.66      0.68      0.63       350

micro avg                              0.68


### BIC 20

In [141]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven',
                'health_oriented', 'adjustment', 'reward_bias', 'cheerful', 'leisure_oriented',
                'thinking_style', 'aggressive', 'organized', 'self_assured', 'religion_oriented',
                'cautious', 'impulsive', 'disciplined', 'type_a', 'liberal']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [96]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6890243902439025.
ovo_lr: Test f1 micro is 0.6857142857142857
ovo_svc: Cross validation f1 micro is 0.6841764528756398.
ovo_svc: Test f1 micro is 0.6771428571428572
ovo_rf: Cross validation f1 micro is 0.6792833483890394.
ovo_rf: Test f1 micro is 0.66
ovo_nb: Cross validation f1 micro is 0.6054049984944294.
ovo_nb: Test f1 micro is 0.6
ovr_lr: Cross validation f1 micro is 0.6915236374585968.
ovr_lr: Test f1 micro is 0.6942857142857143
ovr_svc: Cross validation f1 micro is 0.681737428485396.
ovr_svc: Test f1 micro is 0.6885714285714286
ovr_rf: Cross validation f1 micro is 0.6792532369768141.
ovr_rf: Test f1 micro is 0.68
ovr_nb: Cross validation f1 micro is 0.6349292381812707.
ovr_nb: Test f1 micro is 0.6257142857142857
xgb: Cross validation f1 micro is 0.6719060523938571.
xgb: Test f1 micro is 0.66


In [97]:
mod = onevsrest_lr
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.56      0.18      0.27        50
           1       0.50      0.28      0.36        69
           2       0.73      0.93      0.82       231

    accuracy                           0.69       350
   macro avg       0.60      0.46      0.48       350
weighted avg       0.66      0.69      0.65       350

micro avg                              0.69


### BIC 25

In [151]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven',
                'health_oriented', 'adjustment', 'reward_bias', 'cheerful', 'leisure_oriented',
                'thinking_style', 'aggressive', 'organized', 'self_assured', 'religion_oriented',
                'cautious', 'impulsive', 'disciplined', 'type_a', 'liberal',
                'melancholy', 'neuroticism', 'sexual_focus', 'food_focus', 'cold']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [99]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.687789822342668.
ovo_lr: Test f1 micro is 0.6885714285714286
ovo_svc: Cross validation f1 micro is 0.6891147244805781.
ovo_svc: Test f1 micro is 0.68
ovo_rf: Cross validation f1 micro is 0.673170731707317.
ovo_rf: Test f1 micro is 0.68
ovo_nb: Cross validation f1 micro is 0.622583559168925.
ovo_nb: Test f1 micro is 0.6114285714285714
ovr_lr: Cross validation f1 micro is 0.6890243902439023.
ovr_lr: Test f1 micro is 0.6885714285714286
ovr_svc: Cross validation f1 micro is 0.6755796446853357.
ovr_svc: Test f1 micro is 0.6857142857142857
ovr_rf: Cross validation f1 micro is 0.6817675398976212.
ovr_rf: Test f1 micro is 0.6657142857142857
ovr_nb: Cross validation f1 micro is 0.6349292381812707.
ovr_nb: Test f1 micro is 0.6285714285714286
xgb: Cross validation f1 micro is 0.6583860283047275.
xgb: Test f1 micro is 0.6628571428571428


In [103]:
mod = onevsone_lr
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.60      0.24      0.34        50
           1       0.44      0.26      0.33        69
           2       0.73      0.91      0.81       231

    accuracy                           0.69       350
   macro avg       0.59      0.47      0.49       350
weighted avg       0.65      0.69      0.65       350

micro avg                              0.69


### BIC 30

In [153]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven',
                'health_oriented', 'adjustment', 'reward_bias', 'cheerful', 'leisure_oriented',
                'thinking_style', 'aggressive', 'organized', 'self_assured', 'religion_oriented',
                'cautious', 'impulsive', 'disciplined', 'type_a', 'liberal',
                'melancholy', 'neuroticism', 'sexual_focus', 'food_focus', 'cold',
                'friend_focus', 'agreeableness', 'generous', 'genuine', 'self_conscious']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [105]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6803824149352604.
ovo_lr: Test f1 micro is 0.6828571428571428
ovo_svc: Cross validation f1 micro is 0.6866907557964469.
ovo_svc: Test f1 micro is 0.6771428571428572
ovo_rf: Cross validation f1 micro is 0.6829569406805178.
ovo_rf: Test f1 micro is 0.6857142857142857
ovo_nb: Cross validation f1 micro is 0.6090484793736827.
ovo_nb: Test f1 micro is 0.6257142857142857
ovr_lr: Cross validation f1 micro is 0.6841312857573019.
ovr_lr: Test f1 micro is 0.6885714285714286
ovr_svc: Cross validation f1 micro is 0.6854411321890997.
ovr_svc: Test f1 micro is 0.68
ovr_rf: Cross validation f1 micro is 0.6731105088828666.
ovr_rf: Test f1 micro is 0.6771428571428572
ovr_nb: Cross validation f1 micro is 0.642186088527552.
ovr_nb: Test f1 micro is 0.6257142857142857
xgb: Cross validation f1 micro is 0.6670280036133694.
xgb: Test f1 micro is 0.6542857142857142


In [106]:
mod = onevsrest_lr
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.65      0.26      0.37        50
           1       0.45      0.30      0.36        69
           2       0.73      0.90      0.81       231

    accuracy                           0.69       350
   macro avg       0.61      0.49      0.51       350
weighted avg       0.66      0.69      0.66       350

micro avg                              0.69


### BIC 35

In [107]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven',
                'health_oriented', 'adjustment', 'reward_bias', 'cheerful', 'leisure_oriented',
                'thinking_style', 'aggressive', 'organized', 'self_assured', 'religion_oriented',
                'cautious', 'impulsive', 'disciplined', 'type_a', 'liberal',
                'melancholy', 'neuroticism', 'sexual_focus', 'food_focus', 'cold',
                'friend_focus', 'agreeableness', 'generous', 'genuine', 'self_conscious',
                'emotionally_aware', 'artistic', 'ambitious', 'adventurous', 'anxious']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [108]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6841011743450769.
ovo_lr: Test f1 micro is 0.6771428571428572
ovo_svc: Cross validation f1 micro is 0.687940379403794.
ovo_svc: Test f1 micro is 0.68
ovo_rf: Cross validation f1 micro is 0.684206564287865.
ovo_rf: Test f1 micro is 0.6714285714285714
ovo_nb: Cross validation f1 micro is 0.6017163504968382.
ovo_nb: Test f1 micro is 0.6085714285714285
ovr_lr: Cross validation f1 micro is 0.6816470942487202.
ovr_lr: Test f1 micro is 0.6742857142857143
ovr_svc: Cross validation f1 micro is 0.6817524841915086.
ovr_svc: Test f1 micro is 0.6885714285714286
ovr_rf: Cross validation f1 micro is 0.6755947003914482.
ovr_rf: Test f1 micro is 0.6771428571428572
ovr_nb: Cross validation f1 micro is 0.6299457994579947.
ovr_nb: Test f1 micro is 0.62
xgb: Cross validation f1 micro is 0.6670129479072567.
xgb: Test f1 micro is 0.6657142857142857


In [109]:
mod = onevsrest_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.64      0.18      0.28        50
           1       0.44      0.23      0.30        69
           2       0.72      0.94      0.81       231

    accuracy                           0.69       350
   macro avg       0.60      0.45      0.47       350
weighted avg       0.65      0.69      0.64       350

micro avg                              0.69


### BIC 40

In [110]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven',
                'health_oriented', 'adjustment', 'reward_bias', 'cheerful', 'leisure_oriented',
                'thinking_style', 'aggressive', 'organized', 'self_assured', 'religion_oriented',
                'cautious', 'impulsive', 'disciplined', 'type_a', 'liberal',
                'melancholy', 'neuroticism', 'sexual_focus', 'food_focus', 'cold',
                'friend_focus', 'agreeableness', 'generous', 'genuine', 'self_conscious',
                'emotionally_aware', 'artistic', 'ambitious', 'adventurous', 'anxious',
                'stressed', 'work_oriented', 'depression', 'family_oriented', 'insecure']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [111]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6829117735621801.
ovo_lr: Test f1 micro is 0.6828571428571428
ovo_svc: Cross validation f1 micro is 0.6866757000903341.
ovo_svc: Test f1 micro is 0.6771428571428572
ovo_rf: Cross validation f1 micro is 0.6682926829268292.
ovo_rf: Test f1 micro is 0.6714285714285714
ovo_nb: Cross validation f1 micro is 0.578319783197832.
ovo_nb: Test f1 micro is 0.62
ovr_lr: Cross validation f1 micro is 0.6804426377597108.
ovr_lr: Test f1 micro is 0.6628571428571428
ovr_svc: Cross validation f1 micro is 0.6817675398976212.
ovr_svc: Test f1 micro is 0.6885714285714286
ovr_rf: Cross validation f1 micro is 0.679298404095152.
ovr_rf: Test f1 micro is 0.6657142857142857
ovr_nb: Cross validation f1 micro is 0.6311803673592292.
ovr_nb: Test f1 micro is 0.6228571428571429
xgb: Cross validation f1 micro is 0.6767991568804577.
xgb: Test f1 micro is 0.6628571428571428


In [112]:
mod = onevsrest_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))
print('macro avg                              {}'.format(round(f1_score(y_test, test_pred, average='macro'),2)))

              precision    recall  f1-score   support

           0       0.64      0.18      0.28        50
           1       0.46      0.23      0.31        69
           2       0.72      0.94      0.81       231

    accuracy                           0.69       350
   macro avg       0.61      0.45      0.47       350
weighted avg       0.66      0.69      0.64       350

micro avg                              0.69
macro avg                              0.47


### BIC 45

In [113]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven',
                'health_oriented', 'adjustment', 'reward_bias', 'cheerful', 'leisure_oriented',
                'thinking_style', 'aggressive', 'organized', 'self_assured', 'religion_oriented',
                'cautious', 'impulsive', 'disciplined', 'type_a', 'liberal',
                'melancholy', 'neuroticism', 'sexual_focus', 'food_focus', 'cold',
                'friend_focus', 'agreeableness', 'generous', 'genuine', 'self_conscious',
                'emotionally_aware', 'artistic', 'ambitious', 'adventurous', 'anxious',
                'stressed', 'work_oriented', 'depression', 'family_oriented', 'insecure',
                'happiness', 'conscientiousness', 'dutiful', 'persuasive', 'cooperative']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [114]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6780186690755796.
ovo_lr: Test f1 micro is 0.6771428571428572
ovo_svc: Cross validation f1 micro is 0.6866907557964469.
ovo_svc: Test f1 micro is 0.6857142857142857
ovo_rf: Cross validation f1 micro is 0.6830021077988557.
ovo_rf: Test f1 micro is 0.6714285714285714
ovo_nb: Cross validation f1 micro is 0.6090785907859079.
ovo_nb: Test f1 micro is 0.6285714285714286
ovr_lr: Cross validation f1 micro is 0.6718157181571816.
ovr_lr: Test f1 micro is 0.6714285714285714
ovr_svc: Cross validation f1 micro is 0.6719060523938574.
ovr_svc: Test f1 micro is 0.6885714285714286
ovr_rf: Cross validation f1 micro is 0.6707166516109605.
ovr_rf: Test f1 micro is 0.6685714285714286
ovr_nb: Cross validation f1 micro is 0.6238331827762723.
ovr_nb: Test f1 micro is 0.6228571428571429
xgb: Cross validation f1 micro is 0.6707015959048478.
xgb: Test f1 micro is 0.68


In [116]:
mod = onevsrest_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.67      0.16      0.26        50
           1       0.47      0.25      0.32        69
           2       0.72      0.94      0.81       231

    accuracy                           0.69       350
   macro avg       0.62      0.45      0.46       350
weighted avg       0.66      0.69      0.64       350

micro avg                              0.69


### BIC 50

In [117]:
BIC_features = ['netspeak_focus', 'money_oriented', 'workhorse', 'active', 'power_driven',
                'health_oriented', 'adjustment', 'reward_bias', 'cheerful', 'leisure_oriented',
                'thinking_style', 'aggressive', 'organized', 'self_assured', 'religion_oriented',
                'cautious', 'impulsive', 'disciplined', 'type_a', 'liberal',
                'melancholy', 'neuroticism', 'sexual_focus', 'food_focus', 'cold',
                'friend_focus', 'agreeableness', 'generous', 'genuine', 'self_conscious',
                'emotionally_aware', 'artistic', 'ambitious', 'adventurous', 'anxious',
                'stressed', 'work_oriented', 'depression', 'family_oriented', 'insecure',
                'happiness', 'conscientiousness', 'dutiful', 'persuasive', 'cooperative',
                'intellectual', 'humble', 'assertive', 'body_focus', 'friendly']

X_train = train[BIC_features]
X_test = test[BIC_features]

In [118]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6792833483890395.
ovo_lr: Test f1 micro is 0.66
ovo_svc: Cross validation f1 micro is 0.6891297801866907.
ovo_svc: Test f1 micro is 0.6857142857142857
ovo_rf: Cross validation f1 micro is 0.6830021077988557.
ovo_rf: Test f1 micro is 0.6714285714285714
ovo_nb: Cross validation f1 micro is 0.6078891900030111.
ovo_nb: Test f1 micro is 0.62
ovr_lr: Cross validation f1 micro is 0.6829268292682927.
ovr_lr: Test f1 micro is 0.6571428571428571
ovr_svc: Cross validation f1 micro is 0.6756398675097863.
ovr_svc: Test f1 micro is 0.68
ovr_rf: Cross validation f1 micro is 0.6719813309244203.
ovr_rf: Test f1 micro is 0.6742857142857143
ovr_nb: Cross validation f1 micro is 0.6189099668774466.
ovr_nb: Test f1 micro is 0.62
xgb: Cross validation f1 micro is 0.6730954531767539.
xgb: Test f1 micro is 0.6742857142857143


In [119]:
mod = onevsone_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.86      0.12      0.21        50
           1       0.47      0.23      0.31        69
           2       0.71      0.94      0.81       231

    accuracy                           0.69       350
   macro avg       0.68      0.43      0.44       350
weighted avg       0.68      0.69      0.62       350

micro avg                              0.69


### Model with Backward Feaure Selection using all 29 AIC features

In [120]:
AIC_features = ['netspeak_focus', 'liberal', 'self_assured', 'organized', 'type_a',
                'cheerful', 'disciplined', 'neuroticism', 'aggressive', 'adjustment',
                'religion_oriented', 'food_focus', 'generous', 'sexual_focus', 'thinking_style',
                'agreeableness', 'money_oriented', 'melancholy', 'workhorse', 'reward_bias',
                'self_conscious', 'leisure_oriented', 'impulsive', 'cautious', 'active',
                'power_driven', 'genuine', 'cold', 'health_oriented']

X_train = train[AIC_features]
X_test = test[AIC_features]

In [121]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.684056007226739.
ovo_lr: Test f1 micro is 0.68
ovo_svc: Cross validation f1 micro is 0.6854561878952123.
ovo_svc: Test f1 micro is 0.6857142857142857
ovo_rf: Cross validation f1 micro is 0.6829719963866305.
ovo_rf: Test f1 micro is 0.6828571428571428
ovo_nb: Cross validation f1 micro is 0.602905751279735.
ovo_nb: Test f1 micro is 0.6228571428571429
ovr_lr: Cross validation f1 micro is 0.6865853658536585.
ovr_lr: Test f1 micro is 0.68
ovr_svc: Cross validation f1 micro is 0.6817675398976212.
ovr_svc: Test f1 micro is 0.6857142857142857
ovr_rf: Cross validation f1 micro is 0.6767991568804576.
ovr_rf: Test f1 micro is 0.6714285714285714
ovr_nb: Cross validation f1 micro is 0.6446552243300211.
ovr_nb: Test f1 micro is 0.6257142857142857
xgb: Cross validation f1 micro is 0.669482083709726.
xgb: Test f1 micro is 0.6542857142857142


In [123]:
mod = onevsone_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.86      0.12      0.21        50
           1       0.45      0.22      0.29        69
           2       0.71      0.95      0.81       231

    accuracy                           0.69       350
   macro avg       0.67      0.43      0.44       350
weighted avg       0.68      0.69      0.62       350

micro avg                              0.69


### AIC 30

In [124]:
AIC_features = ['netspeak_focus', 'liberal', 'self_assured', 'organized', 'type_a',
                'cheerful', 'disciplined', 'neuroticism', 'aggressive', 'adjustment',
                'religion_oriented', 'food_focus', 'generous', 'sexual_focus', 'thinking_style',
                'agreeableness', 'money_oriented', 'melancholy', 'workhorse', 'reward_bias',
                'self_conscious', 'leisure_oriented', 'impulsive', 'cautious', 'active',
                'power_driven', 'genuine', 'cold', 'health_oriented', 'friend_focus']

X_train = train[AIC_features]
X_test = test[AIC_features]

In [125]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6803824149352604.
ovo_lr: Test f1 micro is 0.6828571428571428
ovo_svc: Cross validation f1 micro is 0.6866907557964469.
ovo_svc: Test f1 micro is 0.6771428571428572
ovo_rf: Cross validation f1 micro is 0.6768142125865702.
ovo_rf: Test f1 micro is 0.68
ovo_nb: Cross validation f1 micro is 0.6090484793736827.
ovo_nb: Test f1 micro is 0.6257142857142857
ovr_lr: Cross validation f1 micro is 0.6841312857573019.
ovr_lr: Test f1 micro is 0.6885714285714286
ovr_svc: Cross validation f1 micro is 0.6854411321890997.
ovr_svc: Test f1 micro is 0.68
ovr_rf: Cross validation f1 micro is 0.6780487804878048.
ovr_rf: Test f1 micro is 0.6685714285714286
ovr_nb: Cross validation f1 micro is 0.642186088527552.
ovr_nb: Test f1 micro is 0.6257142857142857
xgb: Cross validation f1 micro is 0.6682625715146041.
xgb: Test f1 micro is 0.6542857142857142


In [126]:
mod = onevsrest_lr
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.65      0.26      0.37        50
           1       0.45      0.30      0.36        69
           2       0.73      0.90      0.81       231

    accuracy                           0.69       350
   macro avg       0.61      0.49      0.51       350
weighted avg       0.66      0.69      0.66       350

micro avg                              0.69


### AIC 35

In [127]:
AIC_features = ['netspeak_focus', 'liberal', 'self_assured', 'organized', 'type_a',
                'cheerful', 'disciplined', 'neuroticism', 'aggressive', 'adjustment',
                'religion_oriented', 'food_focus', 'generous', 'sexual_focus', 'thinking_style',
                'agreeableness', 'money_oriented', 'melancholy', 'workhorse', 'reward_bias',
                'self_conscious', 'leisure_oriented', 'impulsive', 'cautious', 'active',
                'power_driven', 'genuine', 'cold', 'health_oriented', 'friend_focus',
                'emotionally_aware', 'artistic', 'ambitious', 'adventurous', 'anxious']

X_train = train[AIC_features]
X_test = test[AIC_features]

In [128]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6841011743450769.
ovo_lr: Test f1 micro is 0.6771428571428572
ovo_svc: Cross validation f1 micro is 0.687940379403794.
ovo_svc: Test f1 micro is 0.68
ovo_rf: Cross validation f1 micro is 0.6805329719963866.
ovo_rf: Test f1 micro is 0.6885714285714286
ovo_nb: Cross validation f1 micro is 0.6017163504968382.
ovo_nb: Test f1 micro is 0.6085714285714285
ovr_lr: Cross validation f1 micro is 0.6816470942487202.
ovr_lr: Test f1 micro is 0.6742857142857143
ovr_svc: Cross validation f1 micro is 0.6817524841915086.
ovr_svc: Test f1 micro is 0.6885714285714286
ovr_rf: Cross validation f1 micro is 0.6731556760012044.
ovr_rf: Test f1 micro is 0.6742857142857143
ovr_nb: Cross validation f1 micro is 0.6299457994579947.
ovr_nb: Test f1 micro is 0.62
xgb: Cross validation f1 micro is 0.6682174043962661.
xgb: Test f1 micro is 0.6685714285714286


In [129]:
mod = onevsrest_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.64      0.18      0.28        50
           1       0.44      0.23      0.30        69
           2       0.72      0.94      0.81       231

    accuracy                           0.69       350
   macro avg       0.60      0.45      0.47       350
weighted avg       0.65      0.69      0.64       350

micro avg                              0.69


### AIC 40

In [130]:
AIC_features = ['netspeak_focus', 'liberal', 'self_assured', 'organized', 'type_a',
                'cheerful', 'disciplined', 'neuroticism', 'aggressive', 'adjustment',
                'religion_oriented', 'food_focus', 'generous', 'sexual_focus', 'thinking_style',
                'agreeableness', 'money_oriented', 'melancholy', 'workhorse', 'reward_bias',
                'self_conscious', 'leisure_oriented', 'impulsive', 'cautious', 'active',
                'power_driven', 'genuine', 'cold', 'health_oriented', 'friend_focus',
                'emotionally_aware', 'artistic', 'ambitious', 'adventurous', 'anxious',
                'stressed', 'work_oriented', 'depression', 'family_oriented', 'insecure']

X_train = train[AIC_features]
X_test = test[AIC_features]

In [131]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6829117735621801.
ovo_lr: Test f1 micro is 0.6828571428571428
ovo_svc: Cross validation f1 micro is 0.6866757000903341.
ovo_svc: Test f1 micro is 0.6771428571428572
ovo_rf: Cross validation f1 micro is 0.6756097560975609.
ovo_rf: Test f1 micro is 0.6771428571428572
ovo_nb: Cross validation f1 micro is 0.578319783197832.
ovo_nb: Test f1 micro is 0.62
ovr_lr: Cross validation f1 micro is 0.6804426377597108.
ovr_lr: Test f1 micro is 0.6628571428571428
ovr_svc: Cross validation f1 micro is 0.6817675398976212.
ovr_svc: Test f1 micro is 0.6885714285714286
ovr_rf: Cross validation f1 micro is 0.6842517314062029.
ovr_rf: Test f1 micro is 0.68
ovr_nb: Cross validation f1 micro is 0.6311803673592292.
ovr_nb: Test f1 micro is 0.6228571428571429
xgb: Cross validation f1 micro is 0.6755947003914483.
xgb: Test f1 micro is 0.6628571428571428


In [132]:
mod = onevsrest_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.64      0.18      0.28        50
           1       0.46      0.23      0.31        69
           2       0.72      0.94      0.81       231

    accuracy                           0.69       350
   macro avg       0.61      0.45      0.47       350
weighted avg       0.66      0.69      0.64       350

micro avg                              0.69


### AIC 45

In [133]:
AIC_features = ['netspeak_focus', 'liberal', 'self_assured', 'organized', 'type_a',
                'cheerful', 'disciplined', 'neuroticism', 'aggressive', 'adjustment',
                'religion_oriented', 'food_focus', 'generous', 'sexual_focus', 'thinking_style',
                'agreeableness', 'money_oriented', 'melancholy', 'workhorse', 'reward_bias',
                'self_conscious', 'leisure_oriented', 'impulsive', 'cautious', 'active',
                'power_driven', 'genuine', 'cold', 'health_oriented', 'friend_focus',
                'emotionally_aware', 'artistic', 'ambitious', 'adventurous', 'anxious',
                'stressed', 'work_oriented', 'depression', 'family_oriented', 'insecure',
                'happiness', 'conscientiousness', 'dutiful', 'persuasive', 'cooperative']

X_train = train[AIC_features]
X_test = test[AIC_features]

In [134]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6780186690755796.
ovo_lr: Test f1 micro is 0.6771428571428572
ovo_svc: Cross validation f1 micro is 0.6866907557964469.
ovo_svc: Test f1 micro is 0.6857142857142857
ovo_rf: Cross validation f1 micro is 0.6731857874134296.
ovo_rf: Test f1 micro is 0.6771428571428572
ovo_nb: Cross validation f1 micro is 0.6090785907859079.
ovo_nb: Test f1 micro is 0.6285714285714286
ovr_lr: Cross validation f1 micro is 0.6718157181571816.
ovr_lr: Test f1 micro is 0.6714285714285714
ovr_svc: Cross validation f1 micro is 0.6719060523938574.
ovr_svc: Test f1 micro is 0.6885714285714286
ovr_rf: Cross validation f1 micro is 0.6817976513098463.
ovr_rf: Test f1 micro is 0.6828571428571428
ovr_nb: Cross validation f1 micro is 0.6238331827762723.
ovr_nb: Test f1 micro is 0.6228571428571429
xgb: Cross validation f1 micro is 0.6669978922011441.
xgb: Test f1 micro is 0.6828571428571428


In [137]:
mod = onevsrest_svc
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.67      0.16      0.26        50
           1       0.47      0.25      0.32        69
           2       0.72      0.94      0.81       231

    accuracy                           0.69       350
   macro avg       0.62      0.45      0.46       350
weighted avg       0.66      0.69      0.64       350

micro avg                              0.69


### AIC 50

In [138]:
AIC_features = ['netspeak_focus', 'liberal', 'self_assured', 'organized', 'type_a',
                'cheerful', 'disciplined', 'neuroticism', 'aggressive', 'adjustment',
                'religion_oriented', 'food_focus', 'generous', 'sexual_focus', 'thinking_style',
                'agreeableness', 'money_oriented', 'melancholy', 'workhorse', 'reward_bias',
                'self_conscious', 'leisure_oriented', 'impulsive', 'cautious', 'active',
                'power_driven', 'genuine', 'cold', 'health_oriented', 'friend_focus',
                'emotionally_aware', 'artistic', 'ambitious', 'adventurous', 'anxious',
                'stressed', 'work_oriented', 'depression', 'family_oriented', 'insecure',
                'happiness', 'conscientiousness', 'dutiful', 'persuasive', 'cooperative',
                'intellectual', 'humble', 'assertive', 'body_focus', 'friendly']

X_train = train[AIC_features]
X_test = test[AIC_features]

In [139]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation f1 micro is 0.6792833483890395.
ovo_lr: Test f1 micro is 0.66
ovo_svc: Cross validation f1 micro is 0.6891297801866907.
ovo_svc: Test f1 micro is 0.6857142857142857
ovo_rf: Cross validation f1 micro is 0.6694519722975008.
ovo_rf: Test f1 micro is 0.6942857142857143
ovo_nb: Cross validation f1 micro is 0.6078891900030111.
ovo_nb: Test f1 micro is 0.62
ovr_lr: Cross validation f1 micro is 0.6829268292682927.
ovr_lr: Test f1 micro is 0.6571428571428571
ovr_svc: Cross validation f1 micro is 0.6756398675097863.
ovr_svc: Test f1 micro is 0.68
ovr_rf: Cross validation f1 micro is 0.6854712436013248.
ovr_rf: Test f1 micro is 0.6714285714285714
ovr_nb: Cross validation f1 micro is 0.6189099668774466.
ovr_nb: Test f1 micro is 0.62
xgb: Cross validation f1 micro is 0.6743149653718759.
xgb: Test f1 micro is 0.6742857142857143


In [140]:
mod = onevsone_rf
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.53      0.20      0.29        50
           1       0.49      0.29      0.36        69
           2       0.73      0.92      0.82       231

    accuracy                           0.69       350
   macro avg       0.58      0.47      0.49       350
weighted avg       0.66      0.69      0.65       350

micro avg                              0.69


### F Test at p <= 0.01 Cutoff

In [100]:
features = ['netspeak_focus', 'cheerful', 'cooperative', 'aggressive', 'religion_oriented',
            'depression', 'ambitious', 'self_conscious', 'active', 'power_driven',
            'genuine', 'anxious', 'health_oriented', 'work_oriented']

X_train = train[features]
X_test = test[features]

In [101]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation accuracy is 0.6965310914186169.
ovo_lr: Test accuracy is 0.7257142857142858
ovo_svc: Cross validation accuracy is 0.702688782852382.
ovo_svc: Test accuracy is 0.7085714285714285
ovo_rf: Cross validation accuracy is 0.7063621904112701.
ovo_rf: Test accuracy is 0.7085714285714285
ovo_nb: Cross validation accuracy is 0.6031507990608195.
ovo_nb: Test accuracy is 0.6257142857142857
ovr_lr: Cross validation accuracy is 0.6953040975535862.
ovr_lr: Test accuracy is 0.7314285714285714
ovr_svc: Cross validation accuracy is 0.703885480572597.
ovr_svc: Test accuracy is 0.7171428571428572
ovr_rf: Cross validation accuracy is 0.711262591835189.
ovr_rf: Test accuracy is 0.7057142857142857
ovr_nb: Cross validation accuracy is 0.6068166325835037.
ovr_nb: Test accuracy is 0.6428571428571429
xgb: Cross validation accuracy is 0.7002347951223207.
xgb: Test accuracy is 0.7085714285714285


In [102]:
mod = onevsrest_lr
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           1       0.78      0.95      0.85       231
           3       0.58      0.36      0.45        69
           4       0.48      0.24      0.32        50

    accuracy                           0.73       350
   macro avg       0.61      0.52      0.54       350
weighted avg       0.70      0.73      0.70       350

micro avg                              0.73


In [103]:
# training set performance
train_pred = modfit.predict(X_train)
print(classification_report(y_train, train_pred))
print('micro avg                              {}'.format(round(f1_score(y_train, train_pred, average='micro'),2)))

              precision    recall  f1-score   support

           1       0.76      0.92      0.83       538
           3       0.58      0.44      0.50       160
           4       0.49      0.18      0.26       116

    accuracy                           0.72       814
   macro avg       0.61      0.51      0.53       814
weighted avg       0.68      0.72      0.68       814

micro avg                              0.72


In [89]:
print('The accuracy on the test set is 0.54 using one versus rest SVC.')

The accuracy on the test set is 0.54 using one versus rest SVC.


### F Test at p <= 0.001 Cutoff

In [104]:
features = ['netspeak_focus', 'active', 'power_driven', 'health_oriented']

X_train = train[features]
X_test = test[features]

In [105]:
for mod, modname in zip([onevsone_lr, onevsone_svc, onevsone_rf, onevsone_nb, onevsrest_lr, onevsrest_svc, onevsrest_rf,
                         onevsrest_nb, xgb],
                        ['ovo_lr', 'ovo_svc', 'ovo_rf', 'ovo_nb', 'ovr_lr', 'ovr_svc', 'ovr_rf', 'ovr_nb', 'xgb']):
    
    score = cross_val_score(mod, X_train, y_train, cv=10, scoring='f1_micro')
    print('{}: Cross validation f1 micro is {}.'.format(modname, score.mean()))
    
    modfit = mod.fit(X_train, y_train)
    test_pred = modfit.predict(X_test)
    print('{}: Test f1 micro is {}'.format(modname, f1_score(y_test, test_pred, average='micro')))

ovo_lr: Cross validation accuracy is 0.6744679239566765.
ovo_lr: Test accuracy is 0.6771428571428572
ovo_svc: Cross validation accuracy is 0.6793607513443913.
ovo_svc: Test accuracy is 0.6828571428571428
ovo_rf: Cross validation accuracy is 0.6744452018480649.
ovo_rf: Test accuracy is 0.6714285714285714
ovo_nb: Cross validation accuracy is 0.6486328864651973.
ovo_nb: Test accuracy is 0.68
ovr_lr: Cross validation accuracy is 0.6769143376505339.
ovr_lr: Test accuracy is 0.6714285714285714
ovr_svc: Cross validation accuracy is 0.6904036961296675.
ovr_svc: Test accuracy is 0.68
ovr_rf: Cross validation accuracy is 0.6744527758842687.
ovr_rf: Test accuracy is 0.66
ovr_nb: Cross validation accuracy is 0.6523138680602893.
ovr_nb: Test accuracy is 0.6771428571428572
xgb: Cross validation accuracy is 0.6646141028554116.
xgb: Test accuracy is 0.7028571428571428


In [106]:
mod = xgb
modfit = mod.fit(X_train, y_train)
test_pred = modfit.predict(X_test)
print(classification_report(y_test, test_pred))
print('micro avg                              {}'.format(round(f1_score(y_test, test_pred, average='micro'),2)))

              precision    recall  f1-score   support

           1       0.74      0.92      0.82       231
           3       0.53      0.36      0.43        69
           4       0.57      0.16      0.25        50

    accuracy                           0.70       350
   macro avg       0.61      0.48      0.50       350
weighted avg       0.67      0.70      0.66       350

micro avg                              0.7


In [93]:
print('The accuracy on the test set is 0.55 using one versus one logistic regression.')

The accuracy on the test set is 0.55 using one versus one logistic regression.
